In [1]:
from glob import glob
import xarray as xr
import pandas as pd
import numpy as np

from scipy.special import gamma
from scipy.integrate import quad
from scipy.stats import gaussian_kde

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.colors as mcolors

import time

import snowScatt
from pytmatrix import tmatrix, tmatrix_aux, scatter, radar, refractive

ar = 1000.0*np.pi/6.0
br = 3.0

Dice = np.logspace(-6, -1.1, 100) # limit for W band refl

def tm_reflectivity(size, wl, n, ar=1.0):
    scatt = tmatrix.Scatterer(radius=0.5e3*size, # conversion to millimeter radius
                              wavelength=wl*1.0e3, # conversion to millimeters
                              m=n,
                              axis_ratio=1.0/ar)
    scatt.set_geometry(tmatrix_aux.geom_vert_back)
    return radar.radar_xsect(scatt) # mm**2 ... need to integrate and multiply by Rayleigh factor

a_graupel = np.linspace(21, 472, 1000)
a_partrimed = np.linspace(0.0121, 0.024, 1000)
z_tm_graupel = pd.DataFrame(index=a_graupel, columns=Dice)
z_tm_partrimed = pd.DataFrame(index=a_partrimed, columns=Dice)


start = time.time()
wl = tmatrix_aux.wl_X*1.0e-3 # back to meters
for i in Dice:
    for j in a_graupel:
        z_tm_graupel.loc[j, i] = tm_reflectivity(i, wl, refractive.mi(wl*1.0e3, 1.0e-3*j*6.0/np.pi))
for i in Dice:
    for j in a_partrimed:
        z_tm_partrimed.loc[j, i] = tm_reflectivity(i, wl, refractive.mi(wl*1.0e3, 1.0e-3*j*6.0*i**(-1.1)/(np.pi*0.6)), 0.6)
z_tm_graupel.to_csv('z_tm_graupel_X.csv')
z_tm_partrimed.to_csv('z_tm_partrimed_X.csv')
print(time.time() - start )

start = time.time()
wl = tmatrix_aux.wl_Ka*1.0e-3 # back to meters
for i in Dice:
    for j in a_graupel:
        z_tm_graupel.loc[j, i] = tm_reflectivity(i, wl, refractive.mi(wl*1.0e3, 1.0e-3*j*6.0/np.pi))
for i in Dice:
    for j in a_partrimed:
        z_tm_partrimed.loc[j, i] = tm_reflectivity(i, wl, refractive.mi(wl*1.0e3, 1.0e-3*j*6.0*i**(-1.1)/(np.pi*0.6)), 0.6)
z_tm_graupel.to_csv('z_tm_graupel_Ka.csv')
z_tm_partrimed.to_csv('z_tm_partrimed_Ka.csv')
print(time.time() - start )

start = time.time()
wl = tmatrix_aux.wl_W*1.0e-3 # back to meters
for i in Dice:
    for j in a_graupel:
        z_tm_graupel.loc[j, i] = tm_reflectivity(i, wl, refractive.mi(wl*1.0e3, 1.0e-3*j*6.0/np.pi))
for i in Dice:
    for j in a_partrimed:
        z_tm_partrimed.loc[j, i] = tm_reflectivity(i, wl, refractive.mi(wl*1.0e3, 1.0e-3*j*6.0*i**(-1.1)/(np.pi*0.6)), 0.6)
z_tm_graupel.to_csv('z_tm_graupel_W.csv')
z_tm_partrimed.to_csv('z_tm_partrimed_W.csv')
print(time.time() - start )

#################################################################
# THIS SHOULD REALLY GO FROM CSV TO netCDF to include wl in dims!
#################################################################

1280.2231373786926
5673.133372783661
15034.608710765839


In [2]:
tmatrix_aux.w

AttributeError: module 'pytmatrix.tmatrix_aux' has no attribute 'w'

In [ ]:
z_tm_graupel.to_csv('z_tm_graupel_X.csv')
back = pd.read_csv('z_tm_graupel_X.csv', index_col=0, dtype=np.float64, engine='c')
back.columns=back.columns.astype(np.float)
back.iloc[back.index.get_loc(421, 'nearest')].loc[Dice]